In [ ]:
# Install & Setup
!pip install gdown --quiet
!apt-get install -y unrar > /dev/null

import os, gdown, numpy as np, matplotlib.pyplot as plt, tensorflow as tf
from glob import glob
from tqdm.notebook import tqdm
from tensorflow.keras import layers, models
from IPython import display

# Download & Extract Dataset
file_id = '1IzLpEwDg3_CoLAuqvdWeRbT_syfthKca'  # Your 512x512 dataset
url = f'https://drive.google.com/uc?id={file_id}'
rar_path = 'dataset.rar'
extract_path = './dataset'
print("📥 Downloading dataset...")
gdown.download(url, rar_path, quiet=False, fuzzy=True)
print("📦 Extracting dataset...")
os.makedirs(extract_path, exist_ok=True)
!unrar x -o+ {rar_path} {extract_path}
os.remove(rar_path)
print("✅ Extraction complete!")

# Dataset Preprocessing
IMG_SIZE, BATCH_SIZE = 512, 16
image_paths = glob(os.path.join(extract_path, 'final 512', '*.jpg'), recursive=True)
if not image_paths: raise ValueError("❌ No JPG images found!")

def preprocess_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    return (tf.cast(img, tf.float32) - 127.5) / 127.5

dataset = tf.data.Dataset.from_tensor_slices(image_paths)
dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Generator (512x512 output)
def make_generator_model():
    model = models.Sequential([
        layers.Input(shape=(100,)),
        layers.Dense(8*8*512, use_bias=False), layers.BatchNormalization(), layers.LeakyReLU(),
        layers.Reshape((8, 8, 512)),
        layers.UpSampling2D(), layers.Conv2D(256, 3, padding='same', use_bias=False), layers.BatchNormalization(), layers.LeakyReLU(),  # 16x16
        layers.UpSampling2D(), layers.Conv2D(128, 3, padding='same', use_bias=False), layers.BatchNormalization(), layers.LeakyReLU(),  # 32x32
        layers.UpSampling2D(), layers.Conv2D(128, 3, padding='same', use_bias=False), layers.BatchNormalization(), layers.LeakyReLU(),  # 64x64
        layers.UpSampling2D(), layers.Conv2D(64, 3, padding='same', use_bias=False),  layers.BatchNormalization(), layers.LeakyReLU(),  # 128x128
        layers.UpSampling2D(), layers.Conv2D(64, 3, padding='same', use_bias=False),  layers.BatchNormalization(), layers.LeakyReLU(),  # 256x256
        layers.UpSampling2D(), layers.Conv2D(3, 3, padding='same', activation='tanh')  # 512x512
    ])
    return model

generator = make_generator_model()

# Dummy SpectralNormalization
def SpectralNormalization(layer): return layer

# Discriminator (512x512 input)
def make_discriminator_model():
    model = models.Sequential([
        layers.Input(shape=(512, 512, 3)),
        SpectralNormalization(layers.Conv2D(64, 4, strides=2, padding='same')), layers.LeakyReLU(0.2), layers.Dropout(0.3),
        SpectralNormalization(layers.Conv2D(128, 4, strides=2, padding='same')), layers.LeakyReLU(0.2), layers.Dropout(0.3),
        SpectralNormalization(layers.Conv2D(256, 4, strides=2, padding='same')), layers.LeakyReLU(0.2), layers.Dropout(0.3),
        SpectralNormalization(layers.Conv2D(512, 4, strides=2, padding='same')), layers.LeakyReLU(0.2), layers.Dropout(0.3),
        SpectralNormalization(layers.Conv2D(512, 4, strides=2, padding='same')), layers.LeakyReLU(0.2), layers.Dropout(0.3),
        layers.Flatten(),
        SpectralNormalization(layers.Dense(1))
    ])
    return model

discriminator = make_discriminator_model()

# Losses & Optimizers
def discriminator_loss(real, fake):
    return tf.keras.backend.mean(tf.nn.relu(1.0 - real)) + tf.keras.backend.mean(tf.nn.relu(1.0 + fake))
def generator_loss(fake): return -tf.keras.backend.mean(fake)

gen_opt = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
disc_opt = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

# Training Setup
EPOCHS, noise_dim = 5000, 100
seed = tf.random.normal([16, noise_dim])

@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        gen_imgs = generator(noise, training=True)
        real_out = discriminator(images, training=True)
        fake_out = discriminator(gen_imgs, training=True)
        gen_loss = generator_loss(fake_out)
        disc_loss = discriminator_loss(real_out, fake_out)
    gen_grads = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_grads = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    gen_opt.apply_gradients(zip(gen_grads, generator.trainable_variables))
    disc_opt.apply_gradients(zip(disc_grads, discriminator.trainable_variables))
    return gen_loss, disc_loss

def generate_and_save_images(model, epoch, test_input):
    preds = model(test_input, training=False)
    fig = plt.figure(figsize=(4,4))
    for i in range(preds.shape[0]):
        plt.subplot(4,4,i+1)
        plt.imshow((preds[i] + 1) / 2.0)
        plt.axis('off')
    plt.suptitle(f"Epoch {epoch}")
    plt.tight_layout()
    plt.savefig(f"generated_images_epoch_{epoch}.png")
    plt.show()
    plt.close()

def train(dataset, epochs):
    for epoch in tqdm(range(epochs), desc="Overall Training Progress"):
        gen_losses, disc_losses = [], []
        for batch in tqdm(dataset, desc=f"Epoch {epoch+1}/{epochs}", leave=False):
            gen_loss, disc_loss = train_step(batch)
            gen_losses.append(gen_loss)
            disc_losses.append(disc_loss)
        if (epoch+1) % 50 == 0 or (epoch+1) == epochs:
            avg_gen = np.mean(gen_losses)
            avg_disc = np.mean(disc_losses)
            print(f"\n📊 Epoch {epoch+1} | Gen Loss: {avg_gen:.4f}, Disc Loss: {avg_disc:.4f}")
            display.clear_output(wait=True)
            generate_and_save_images(generator, epoch+1, seed)
        if (epoch+1) % 500 == 0:
            generator.save(f'generator_epoch_{epoch+1}.h5')
            discriminator.save(f'discriminator_epoch_{epoch+1}.h5')
    print("🎉 Training Finished!")
    generate_and_save_images(generator, epochs, seed)
    generator.save('generator_final.h5')
    discriminator.save('discriminator_final.h5')

# Start Training
train(dataset, EPOCHS)


📥 Downloading dataset...


Downloading...
From: https://drive.google.com/uc?id=1IzLpEwDg3_CoLAuqvdWeRbT_syfthKca
To: /content/dataset.rar
100%|██████████| 60.1M/60.1M [00:00<00:00, 144MB/s]


📦 Extracting dataset...

UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from dataset.rar

Extracting  ./dataset/final 512/0.jpg                                      0%  OK 
Extracting  ./dataset/final 512/1.jpg                                      0%  OK 
Extracting  ./dataset/final 512/10.jpg                                     0%  OK 
Extracting  ./dataset/final 512/100.jpg                                    0%  OK 
Extracting  ./dataset/final 512/1000.jpg                                   0%  OK 
Extracting  ./dataset/final 512/1001.jpg                                   0%  OK 
Extracting  ./dataset/final 512/1002.jpg                                   0%  OK 
Extracting  ./dataset/final 512/1003.jpg                                   0%  OK 
Extracting  ./dataset/final 512/1004.jpg                                   0%  OK 
Extracting  ./dataset/final 512/1005.jpg   

Overall Training Progress:   0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 1/5000:   0%|          | 0/101 [00:00<?, ?it/s]

Epoch 2/5000:   0%|          | 0/101 [00:00<?, ?it/s]

Epoch 3/5000:   0%|          | 0/101 [00:00<?, ?it/s]

Epoch 4/5000:   0%|          | 0/101 [00:00<?, ?it/s]

Epoch 5/5000:   0%|          | 0/101 [00:00<?, ?it/s]

Epoch 6/5000:   0%|          | 0/101 [00:00<?, ?it/s]